In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'HNF5QHXIWOKHP4FBJ0A4W32RNEQWY13ZADPNVN1ODETECMCU' # your Foursquare ID
CLIENT_SECRET = 'G4EMPUSWZTQKC4S2LGYO0VLYQTBBWYQDOZVFSA3JRGWILHQZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HNF5QHXIWOKHP4FBJ0A4W32RNEQWY13ZADPNVN1ODETECMCU
CLIENT_SECRET:G4EMPUSWZTQKC4S2LGYO0VLYQTBBWYQDOZVFSA3JRGWILHQZ


In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
def getLocalVenues(names, latitudes, longitudes, section='topPicks', radius=5000):
    LIMIT = 50
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            section,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id'],
            v['venue']['categories'][0]['shortName']) for v in results])
    trending_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    trending_venues.columns = [
        'location', 
        'city_latitude', 
        'city_longitude', 
        'venue_id',
        'venue', 
        'venue_latitude', 
        'venue_longitude', 
        'venue_category',
        'venue_cat_code',
        'venue_cat_short']
    
    return(trending_venues)

In [5]:
pop_data = pd.read_csv('../data/processed/us_city_populations.csv')
top_venues = getLocalVenues(
    names=pop_data.location, 
    latitudes=pop_data.latitude.values,
    longitudes=pop_data.longitude.values
)
top_venues.to_csv('../data/processed/us_city_venues.csv', index=False)

In [6]:
# one hot encoding
venues_onehot = pd.get_dummies(top_venues[['venue_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['location'] = top_venues['location'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_grouped = venues_onehot.groupby('location').mean().reset_index()

In [7]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
# set number of clusters
kclusters = 15

venues_groupedclustering = venues_grouped.drop('location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_groupedclustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([ 0,  8,  4,  8,  7,  9, 12,  1,  0,  7], dtype=int32)

In [42]:
venues_merged = venues_grouped

# add clustering labels
venues_merged['cluster_labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
venues_merged = venues_merged.join(pop_data.set_index('location'), on='location').reset_index(drop=True) # check the last columns!

In [43]:
# create map
latitude, longitude = 38, -97
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['latitude'], venues_merged['longitude'], venues_merged.location, venues_merged['cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters